In [52]:
#part(a)
import pandas as pd

file_path = 'Assignment-1.xlsx'
sheet_name = 'Auto MPG'  

df = pd.read_excel(file_path, sheet_name=sheet_name)

df.columns = df.columns.str.strip()

print("Available columns:", df.columns)

#finding mean of column horsepower
column_name = 'horsepower'

if column_name in df.columns:
    df[column_name] = pd.to_numeric(df[column_name], errors='coerce')
    mean_value = df[column_name].mean()

    print(f"The mean of the column '{column_name}' is: {mean_value}")
else:
    print(f"The column '{column_name}' does not exist in the dataset.")


Available columns: Index(['mpg', 'cylinders', 'displacement', 'horsepower', 'weight',
       'acceleration', 'model year', 'origin'],
      dtype='object')
The mean of the column 'horsepower' is: 104.46938775510205


In [64]:
#replaced blank values with mean

df.replace('?', pd.NA, inplace=True)
df.fillna(mean_value, inplace=True)

df.to_excel('modified_dataset.xlsx', index=False)

row_index = 330  

print("\nParticular Row:\n", df.iloc[row_index])


Particular Row:
 mpg               40.900000
cylinders          4.000000
displacement      85.000000
horsepower       104.469388
weight          1835.000000
acceleration      17.300000
model year        80.000000
origin             2.000000
Name: 330, dtype: float64


In [66]:
# I deleted last row
df = df.iloc[:-1]

df.to_excel('modified_dataset-1.xlsx', index=False)

In [112]:
#perfoemed one hot encodings
import pandas as pd

file_path = 'modified_dataset-1.xlsx'
sheet_name = 'Sheet1' 

df = pd.read_excel(file_path, sheet_name=sheet_name)

columns_to_encode = ['cylinders', 'model year', 'origin']  

encoded_df = pd.get_dummies(df, columns=columns_to_encode)

encoded_df = encoded_df.astype(int)

print(encoded_df)

encoded_df.to_excel('encoded_dataset.xlsx', index=False)


     mpg  displacement  horsepower  weight  acceleration  cylinders_3  \
0     18           307         130    3504            12            0   
1     15           350         165    3693            11            0   
2     18           318         150    3436            11            0   
3     16           304         150    3433            12            0   
4     17           302         140    3449            10            0   
..   ...           ...         ...     ...           ...          ...   
393   27           140          86    2790            15            0   
394   44            97          52    2130            24            0   
395   32           135          84    2295            11            0   
396   28           120          79    2625            18            0   
397   31           119          82    2720            19            0   

     cylinders_4  cylinders_5  cylinders_6  cylinders_8  ...  model year_77  \
0              0            0            0  

In [120]:
#handle outliers/noise
import pandas as pd
import numpy as np

file_path = 'encoded_dataset.xlsx'
sheet_name = 'Sheet1'  

df = pd.read_excel(file_path, sheet_name=sheet_name)

numerical_columns = ['mpg', 'displacement', 'horsepower', 'weight', 'acceleration'] 

def standardize_outliers(df, columns, threshold=2):
    for column in columns:
        if column in df.columns:
            z_scores = (df[column] - df[column].mean()) / df[column].std()
            
            df[column] = df[column].where(np.abs(z_scores) <= threshold, 
                                           df[column].clip(lower=df[column].mean() - threshold * df[column].std(), 
                                                           upper=df[column].mean() + threshold * df[column].std()))
    return df


df_standardized = standardize_outliers(df, numerical_columns)

print(df_standardized)

df_standardized.to_excel('standardized_dataset.xlsx', index=False)

print("\nStandardized dataset saved to 'standardized_dataset.xlsx'.")


           mpg  displacement  horsepower  weight  acceleration  cylinders_3  \
0    18.000000         307.0       130.0  3504.0      12.00000            0   
1    15.000000         350.0       165.0  3693.0      11.00000            0   
2    18.000000         318.0       150.0  3436.0      11.00000            0   
3    16.000000         304.0       150.0  3433.0      12.00000            0   
4    17.000000         302.0       140.0  3449.0      10.00000            0   
..         ...           ...         ...     ...           ...          ...   
393  27.000000         140.0        86.0  2790.0      15.00000            0   
394  38.854329          97.0        52.0  2130.0      20.70122            0   
395  32.000000         135.0        84.0  2295.0      11.00000            0   
396  28.000000         120.0        79.0  2625.0      18.00000            0   
397  31.000000         119.0        82.0  2720.0      19.00000            0   

     cylinders_4  cylinders_5  cylinders_6  cylinde

In [1]:
#part(b) 
import openpyxl

# Load the preprocessed data from part (a)
wb = openpyxl.load_workbook('standardized_dataset.xlsx')
sheet = wb.active

#data of dataset
data = []
for row in sheet.iter_rows(min_row=2, max_col=5, values_only=True): 
    data.append(list(row))


#mean calculation
means = []
for col in range(5):
    col_val = [row[col] for row in data]
    mean = sum(col_val) / len(col_val)
    means.append(mean)


# variance calculation
variances = []
for col in range(5):
    col_val = [row[col] for row in data]
    mean = means[col]
    variance = sum((x - mean) ** 2 for x in col_val) / len(col_val)
    variances.append(variance)

print("Mean for each column:")
print(means)
print("\nVariance for each column:")
print(variances)


Mean for each column:
[23.259427017767706, 192.49924523902115, 103.2754601174472, 2964.2481043270727, 15.18737440385699]

Variance for each column:
[56.724951736258284, 10417.526744596142, 1231.992205408896, 706064.1680495146, 6.693429304115046]


In [7]:
import openpyxl

def read_excel(file_path):
    workbook = openpyxl.load_workbook(file_path)
    sheet = workbook.active
    data = []
    for row in sheet.iter_rows(values_only=True):
        data.append(list(row))
    return data

#mean calculation
def compute_mean(data):
    mean = []
    for col in range(len(data[0])):
        col_sum = sum(row[col] for row in data)
        mean.append(col_sum / len(data))
    return mean

#standard deviation calculation
def compute_std(data, mean):
    std = []
    for col in range(len(data[0])):
        variance_sum = sum((row[col] - mean[col]) ** 2 for row in data)
        std.append((variance_sum / len(data)) ** 0.5)
    return std

#normalized data
def norm_data(data, mean, std):
    norm_data = []
    for row in data:
        norm_row = [(row[col] - mean[col]) / std[col] for col in range(len(row))]
        norm_data.append(norm_row)
    return norm_data


#variance calculation
def compute_variance(norm_data):
    variance = []
    for col in range(len(norm_data[0])):
        variance_sum = sum(row[col] ** 2 for row in norm_data)
        variance.append(variance_sum / len(norm_data))
    return variance

data = read_excel('standardized_dataset.xlsx')

data_selected = [row[:5] for row in data[1:]]  

mean = compute_mean(data_selected)
std = compute_std(data_selected, mean)

norm_data = norm_data(data_selected, mean, std)

variance_norm = compute_variance(norm_data)

print("Variance of each normalized feature:")
for feature, var in zip(data[0][:5], variance_norm): 
    print(f"{feature}: {var}")


Variance of each normalized feature:
mpg: 0.9999999999999999
displacement: 1.0
horsepower: 0.9999999999999999
weight: 0.9999999999999999
acceleration: 0.9999999999999999


In [20]:
#part(d)
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency

df = pd.read_excel('Assignment-1.xlsx')

contingency_table = pd.crosstab(df['cylinders'], df['model year'])

chi2, p_value, dof, expected = chi2_contingency(contingency_table)

print("Contingency Table:")
print(contingency_table)
print("\nChi-square statistic:", chi2)
print("p-value:", p_value)
print("Degrees of freedom:", dof)


alpha = 0.05   #5% level of significance

if p_value < alpha:
    print("\nReject the null hypothesis.")
    print("There is evidence to suggest that the number of cylinders is affected by the model year.")
else:
    print("\nFail to reject the null hypothesis.")
    print("Not enough evidence to suggest that the number of cylinders is affected by the model year.")


print("\nExpected Frequencies:")
print(pd.DataFrame(expected, index=contingency_table.index, columns=contingency_table.columns))

Contingency Table:
model year  70.0     71.0     72.0     73.0     74.0     75.0     76.0     \
cylinders                                                                   
3                 0        0        1        1        0        0        0   
4                 7       13       14       11       15       12       15   
5                 0        0        0        0        0        0        0   
6                 4        8        0        8        7       12       10   
8                18        7       13       20        5        6        9   

model year  77.0     78.0     79.0     80.0     81.0     82.0     99999.0  
cylinders                                                                  
3                 1        0        0        1        0        0        0  
4                14       17       12       24       21       28        1  
5                 0        1        1        1        0        0        0  
6                 5       12        6        2        7      